# As Rigid As Possible

In [1]:
import igl
from meshplot import plot
import numpy as np
import os
import spharapy.trimesh as trimesh
import matplotlib.pyplot as plt
from collections import defaultdict

%matplotlib inline
# root_folder = os.getcwd()

In [2]:
def get_1_ring_neighbour(F):
#     input:
#         F : faces
#     output:
#         neighbours : neighbours of each vertex dict{key : vertex =>  value : sorted list of neighbours}
    
    neighbours = defaultdict(set)
    for i in F:
        a,b,c = i[0],i[1],i[2]
        neighbours[a].add(b)
        neighbours[a].add(c)
        neighbours[b].add(a)
        neighbours[b].add(c)
        neighbours[c].add(a)
        neighbours[c].add(b)
    
    for vertex in neighbours:
        neighbours[vertex] = np.array(sorted(list(neighbours[vertex])))
    return neighbours

In [3]:
def get_1_ring_edges(V, neighbours):
#     input:
#         V : Vertices
#         neighbours : neighbours of each vertex dict{key : vertex =>  value : sorted list of neighbours}
#     output:
#         edges : edges of each vertex dict{key : vertex =>  value : 3 X neighbour[vertex] matrix}

    edges = dict()
    for vertex in neighbours:
        edges[vertex] = (V[vertex] - V[neighbours[vertex]]).T
    return edges

In [4]:
def get_weights(V, F, neighbours, form = 'cotant'):
#     input:
#         V : Vertices
#         F : Faces
#         neighbours : neighbours of each vertex dict{key : vertex =>  value : sorted list of neighbours}
#         form : type of weight (one or cotant)
#     output:
#         D : weight of each edge dict{key : vertex =>  value : list of weights }

    D = dict()
    if form == 'cotant':
        l = igl.cotmatrix(V, F)
        l = l.todense()
        for vertex in neighbours:
            D[vertex] = np.array([l[vertex,nb] for nb in neighbours[vertex]])
    else:
        for vertex in neighbours:
            D[vertex] = np.ones(neighbours[vertex].size)
        
    return D

In [5]:
def get_transformation(P,D,P_):
#     input:
#         P : edges of given mesh dict{key : vertex =>  value : 3 X neighbour[vertex] matrix }
#         D : weight of each edge dict{key : vertex =>  value : list of weights }
#         P_ : edges of deformed mesh dict{key : vertex =>  value : 3 X neighbour[vertex] matrix }
#     output:
#         R : rotation matrix for each vertex dict{key : vertex =>  value : 3 X 3 matrix }

    R = dict()
    for vertex in P:
        S = P[vertex] @ np.diag(D[vertex]) @ (P_[vertex].T)
        U, Sigma, Vh = np.linalg.svd(S)
        R[vertex] = (Vh.T) @ (U.T)
    return R

In [6]:
def get_new_vertices(R, neighbours, given_edgs, weights, L, fix_V, fix_V_indices, var_V_indices):
#     input:
#         R : rotation matrix for each vertex dict{key : vertex =>  value : 3 X 3 matrix }
#         neighbours : neighbours of each vertex dict{key : vertex =>  value : sorted list of neighbours}
#         given_edgs : edges of given mesh dict{key : vertex =>  value : 3 X neighbour[vertex] matrix }
#         weights : weight of each edge dict{key : vertex =>  value : list of weights }
#         L : Laplace beltrami operator
#         fix_V : position of fix vertices
#         fix_V_indices : indices of fix vertices
#         var_V_indices : indices of variable vertices
#     output:
#         V_ : new vertices
    
    b = np.zeros((L.shape[0], 3))
    for vertex in neighbours:
        for i in range(neighbours[vertex].size):
            b[vertex] += (0.5)*weights[vertex][i]*((R[vertex]+R[neighbours[vertex][i]])@given_edgs[vertex][:,i])
    
    temp = L[:,fix_V_indices] @ fix_V
    temp_L = np.delete(L,fix_V_indices,1)
    
    V_ = np.zeros_like(b)
    V_[fix_V_indices] = fix_V
    V_[var_V_indices] = np.linalg.lstsq(temp_L,b-temp,rcond = None)[0]
    
    return V_

In [7]:
def LBO(V,F):
#     input:
#         V : vertices
#         F : Faces
#     output:
#         L : Laplace beltrami operator
    tm = trimesh.TriMesh(F, V)
    L = tm.laplacianmatrix(mode='half_cotangent')
    return L

In [8]:
def Laplacian_editing(V, F, L, fix_V, fix_V_indices, var_V_indices):
#     input:
#         V : vertices
#         F : Faces
#         L : Laplace beltrami operator
#         fix_V : position of fix vertices
#         fix_V_indices : indices of fix vertices
#         var_V_indices : indices of variable vertices
#     output:
#         V_ : new vertices
    delta = L@V
    V_ = np.zeros_like(V)
    V_[fix_V_indices] = fix_V
    delta = delta - L[:,fix_V_indices] @ V_[fix_V_indices]
    
    temp_L = np.delete(L,fix_V_indices,1)
    
    V_[var_V_indices] = np.linalg.lstsq(temp_L,delta,rcond = None)[0]
    return V_

In [9]:
def Rotation(theta,n):
    # Given the angle of rotation and axis of rotation define a rotation matrix
    # Input: 
    #     theta: Angle o rotation
    #     n    : Axis of rotation
    # Output: 
    #     R    : Rotation matrix
    
    # Define skew-symmetric matrix
    Nx = np.zeros([3,3])
    Nx[0,1],Nx[0,2],Nx[1,2]= -n[2],n[1],-n[0]
    Nx[1,0],Nx[2,0],Nx[2,1]=  n[2],-n[1],n[0]
    
    # Rodrigues' rotation formula
    R = np.eye(3) + np.sin(theta)*Nx+ (1-np.cos(theta))*(Nx@Nx)
    
    return R

In [10]:
# you can change this function based on mashes

def get_fix_vertices(V):
#     input:
#         V : vertices
#     output:
#         fix_V : position of fix vertices
#         fix_V_indices : indices of fix vertices
#         var_V_indices : indices of variable vertices
    fix_V_indices = np.arange(98)
    var_V_indices = np.arange(98,V.shape[0])
    Transformation = Rotation(np.pi/2,np.array([0,1,0]))
    fix_V = np.zeros((fix_V_indices.size,3))
    fix_V[fix_V_indices[:49]] = V[fix_V_indices[:49]] @ Transformation
    fix_V[fix_V_indices[49:]] = V[fix_V_indices[49:]]
    return fix_V, fix_V_indices, var_V_indices

In [11]:
# import mesh and visualization
V, F = igl.read_triangle_mesh("bar.obj")
s = np.zeros(V.shape[0])
s[0:98] = 1
plot(V, F, s, shading={"wireframe": True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.768371…

In [12]:
# Laplace beltrami operator
L = LBO(V, F)

In [13]:
# 1 ring neighbours of all vertices
neighbours = get_1_ring_neighbour(F)

In [14]:
# edges between all neighbours
given_edgs = get_1_ring_edges(V, neighbours)  # P

In [15]:
# cotant or unit weights
weights = get_weights(V, F, neighbours) # D

In [16]:
# fix vertices and their position
fix_V, fix_V_indices, var_V_indices = get_fix_vertices(V)

In [17]:
# output using laplace mesh edition and the 1st estimation of deformed mesh
V_ = Laplacian_editing(V, F, L, fix_V, fix_V_indices, var_V_indices)
plot(V_, F, s, shading={"wireframe": True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 9.15…

In [18]:
#iterative method to find deformed mesh
for _ in range(50):
    new_edgs = get_1_ring_edges(V_, neighbours) # P_
    R = get_transformation(given_edgs,weights,new_edgs)
    V_ = get_new_vertices(R, neighbours,given_edgs, weights, L, fix_V, fix_V_indices, var_V_indices)
plot(V_, F, s, shading={"wireframe": True}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-9.536743…